<a href="https://colab.research.google.com/github/geethasrisykam/Traffic_Detection/blob/main/Copy_of_Traffic_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/archive.zip

In [13]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from PIL import Image 
import os 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

    

In [25]:
data = [] 
labels = [] 
classes = 43 
cur_path = os.getcwd()

for i in range(classes): 
  path = os.path.join(cur_path, 'train', str(i))
  images = os.listdir(path)

  for a in images: 
    try: 
      image = Image.open (path+'/'+a)
      image = image.resize((30,30))
      image = np.array(image)
      data.append(image)
      labels.append (i)
    except: 
      print("Error loading image")
data = np.array(data)
labels = np.array(labels) 

In [26]:
print(data.shape, labels.shape)

(39209, 30, 30, 3) (39209,)


In [29]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.35, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

(25485, 30, 30, 3) (13724, 30, 30, 3) (25485,) (13724,)


In [30]:
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))


model.add(Conv2D(filters=32, kernel_size=(2, 2), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(2, 2), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))



Epoch 1/15
797/797 [==============================] - 73s 90ms/step - loss: 3.8337 - accuracy: 0.1122 - val_loss: 1.9341 - val_accuracy: 0.3894
Epoch 2/15
797/797 [==============================] - 71s 89ms/step - loss: 1.9653 - accuracy: 0.3863 - val_loss: 1.2814 - val_accuracy: 0.5964
Epoch 3/15
797/797 [==============================] - 71s 89ms/step - loss: 1.4707 - accuracy: 0.5267 - val_loss: 0.6418 - val_accuracy: 0.7909
Epoch 4/15
797/797 [==============================] - 71s 89ms/step - loss: 0.9862 - accuracy: 0.6827 - val_loss: 0.4417 - val_accuracy: 0.8565
Epoch 5/15
797/797 [==============================] - 71s 89ms/step - loss: 0.7244 - accuracy: 0.7689 - val_loss: 0.3162 - val_accuracy: 0.9065
Epoch 6/15
797/797 [==============================] - 71s 89ms/step - loss: 0.5851 - accuracy: 0.8165 - val_loss: 0.2305 - val_accuracy: 0.9291
Epoch 7/15
797/797 [==============================] - 71s 89ms/step - loss: 0.5260 - accuracy: 0.8430 - val_loss: 0.1424 - val_accuracy:

In [32]:
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)

pred = model.predict_classes(X_test)
print(accuracy_score(labels, pred))
model.save('traffic_classifier.h5')


/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.953285827395091
